## Requirements

As of June 30, 2025, the code works with the following environments

- python: 3.12.11; too new pythons such as 3.13 may not work
- crewai: 0.134.0
- rustc: 1.88.0

```bash
# install rustc, needed for installing crewai which compiles Rust code
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
source $HOME/.cargo/env
```

In [ ]:
# install crewai
! pip install "crewai[tools]"

## Load environment variables from .env file

In [9]:
import os
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()

True

## Create an LLM model

In [10]:
from crewai import LLM

# Read your API key from the environment variable
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Use Gemini 2.5 Pro Experimental model
gemini_llm = LLM(
    model="gemini/gemini-2.5-pro",
    api_key=gemini_api_key,
    temperature=0.0,  # Lower temperature for more consistent results.
)

## Define components

CrewAI applications are built using Tools, Agents, Tasks, and the Crew itself.

### Tools
Tools are capabilities that agents can use to interact with the outside world or perform specific actions


In [12]:
from crewai.tools import BaseTool


# Placeholder tool for fetching customer support data
class CustomerSupportDataTool(BaseTool):
    name: str = "Customer Support Data Fetcher"
    description: str = (
        "Fetches recent customer support interactions, tickets, and feedback. "
        "Returns a summary string."
    )

    def _run(self, argument: str) -> str:
        # In a real scenario, this would query a database or API.
        # For this example, return simulated data.
        print(f"--- Fetching data for query: {argument} ---")
        return """Recent Support Data Summary:
- 50 tickets related to 'login issues'. High resolution time (avg 48h).
- 30 tickets about 'billing discrepancies'. Mostly resolved within 12h.
- 20 tickets on 'feature requests'. Often closed without resolution.
- Frequent feedback mentions 'confusing user interface' for password reset.
- High volume of calls related to 'account verification process'.
- Sentiment analysis shows growing frustration with 'login issues' resolution time.
- Support agent notes indicate difficulty reproducing 'login issues'."""


support_data_tool = CustomerSupportDataTool()

### Agents

Agents are the individual AI workers in your crew. Each agent has a specific role, goal, backstory, assigned llm, and optional tools.

In [13]:
from crewai import Agent

# Agent 1: Data analyst
data_analyst = Agent(
    role="Customer Support Data Analyst",
    goal="Analyze customer support data to identify trends, recurring issues, and key pain points.",
    backstory=(
        """You are an expert data analyst specializing in customer support operations.
        Your strength lies in identifying patterns and quantifying problems from raw support data."""
    ),
    verbose=True,
    allow_delegation=False,  # This agent focuses on its specific task
    tools=[support_data_tool],  # Assign the data fetching tool
    llm=gemini_llm,  # Use the configured Gemini LLM
)

# Agent 2: Process optimizer
process_optimizer = Agent(
    role="Process Optimization Specialist",
    goal="Identify bottlenecks and inefficiencies in current support processes based on the data analysis. Propose actionable improvements.",
    backstory=(
        """You are a specialist in optimizing business processes, particularly in customer support.
        You excel at pinpointing root causes of delays and inefficiencies and suggesting concrete solutions."""
    ),
    verbose=True,
    allow_delegation=False,
    # No tools needed, this agent relies on the context provided by data_analyst.
    llm=gemini_llm,
)

# Agent 3: Report writer
report_writer = Agent(
    role="Executive Report Writer",
    goal="Compile the analysis and improvement suggestions into a concise, clear, and actionable report for the COO.",
    backstory=(
        """You are a skilled writer adept at creating executive summaries and reports.
        You focus on clarity, conciseness, and highlighting the most critical information and recommendations for senior leadership."""
    ),
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm,
)

### Tasks

Tasks define the specific assignments for the agents. Each task has a description, expected_output, and is assigned to an agent. Tasks are run sequentially by default and include the context of the previous task. 

In [14]:
from crewai import Task

# Task 1: Analyze data
analysis_task = Task(
    description=(
        """Fetch and analyze the latest customer support interaction data (tickets, feedback, call logs)
        focusing on the last quarter. Identify the top 3-5 recurring issues, quantify their frequency
        and impact (e.g., resolution time, customer sentiment). Use the Customer Support Data Fetcher tool."""
    ),
    expected_output=(
        """A summary report detailing the key findings from the customer support data analysis, including:
- Top 3-5 recurring issues with frequency.
- Average resolution times for these issues.
- Key customer pain points mentioned in feedback.
- Any notable trends in sentiment or support agent observations."""
    ),
    agent=data_analyst,  # Assign task to the data_analyst agent
)

# Task 2: Identify bottlenecks and suggest improvements
optimization_task = Task(
    description=(
        """Based on the data analysis report provided by the Data Analyst, identify the primary bottlenecks
        in the support processes contributing to the identified issues (especially the top recurring ones).
        Propose 2-3 concrete, actionable process improvements to address these bottlenecks.
        Consider potential impact and ease of implementation."""
    ),
    expected_output=(
        """A concise list identifying the main process bottlenecks (e.g., lack of documentation for agents,
        complex escalation path, UI issues) linked to the key problems.
A list of 2-3 specific, actionable recommendations for process improvement
(e.g., update agent knowledge base, simplify password reset UI, implement proactive monitoring)."""
    ),
    agent=process_optimizer,  # Assign task to the process_optimizer agent
    # This task implicitly uses the output of analysis_task as context
)

# Task 3: Compile COO report
report_task = Task(
    description=(
        """Compile the findings from the Data Analyst and the recommendations from the Process Optimization Specialist
        into a single, concise executive report for the COO. The report should clearly state:
1. The most critical customer support issues identified (with brief data points).
2. The key process bottlenecks causing these issues.
3. The recommended process improvements.
Ensure the report is easy to understand, focuses on actionable insights, and is formatted professionally."""
    ),
    expected_output=(
        """A well-structured executive report (max 1 page) summarizing the critical support issues,
        underlying process bottlenecks, and clear, actionable recommendations for the COO.
        Use clear headings and bullet points."""
    ),
    agent=report_writer,  # Assign task to the report_writer agent
)

### Crew

The Crew brings the agents and tasks together, defining the workflow process (such as "sequential").

In [15]:
from crewai import Crew, Process

# Define the crew with agents, tasks, and process
support_analysis_crew = Crew(
    agents=[data_analyst, process_optimizer, report_writer],
    tasks=[analysis_task, optimization_task, report_task],
    process=Process.sequential,  # Tasks will run sequentially in the order defined
    verbose=True,
)

## Run the Crew

In [16]:
# If you are on macOS and see SSL certificate errors, run this cell once:
import sys
import os

if sys.platform == "darwin":
    try:
        import certifi

        os.environ["SSL_CERT_FILE"] = certifi.where()
    except ImportError:
        print("Installing certifi...")
        # Install certifi if not present
        import subprocess

        subprocess.check_call([sys.executable, "-m", "pip", "install", "certifi"])
        import certifi

        os.environ["SSL_CERT_FILE"] = certifi.where()

# Start the crew's work
print("--- Starting Customer Support Analysis Crew ---")
# The 'inputs' dictionary provides initial context if needed by the first task.
# In this case, the tool simulates data fetching regardless of the input.
result = support_analysis_crew.kickoff(
    inputs={"data_query": "last quarter support data"}
)

print("--- Crew Execution Finished ---")
print("--- Final Report for COO ---")
print(result)

--- Starting Customer Support Analysis Crew ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 18c4ab3c-5646-4187-9ab4-3171a130ae4b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Support Data Analyst                                                                           │
│                                                                                                                 │
│  Task: Fetch and analyze the latest customer support interaction data (tickets, feedback, call logs)            │
│          focusing on the last quarter. Identify the top 3-5 recurring issues, quantify their frequency          │
│          and impact (e.g., resolution time, customer sentiment). Use the Customer Support Data Fetcher tool.    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/ubuntu/work/.pyenv/versions/3.12.11/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

--- Fetching data for query: Fetch and summarize all customer support interactions, tickets, and feedback from the 
last quarter. ---

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Executive Report Writer                                                                                 │
│                                                                                                                 │
│  Task: Compile the findings from the Data Analyst and the recommendations from the Process Optimization         │
│  Specialist                                                                                                     │
│          into a single, concise executive report for the COO. The report should clearly state:                  │
│  1. The most critical customer support issues identified (with brief data points).                              │
│  2. The key process bottlenecks causing these issues.                                                           │
│  3. The recommended process improvements.                                                                       │
│  Ensure the report is easy to understand, focuses on actionable insights, and is formatted professionally.      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Executive Report Writer                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **TO:** Chief Operating Officer                                                                                │
│  **FROM:** Executive Report Writer                                                                              │
│  **DATE:** October 26, 2023                                                                                     │
│  **SUBJECT:** Executive Summary: Analysis and Recommendations for Customer Support Process Improvement          │
│                                                                                                                 │
│  ### **1. Executive Overview**                                                                                  │
│                                                                                                                 │
│  This report summarizes a joint analysis of last quarter's customer support data and underlying operational     │
│  processes. The findings reveal that a few critical, preventable issues are driving the majority of our         │
│  customer support workload and negative sentiment. Our primary challenge is a **48-hour average resolution      │
│  time for login issues**, caused by significant gaps in our support team's diagnostic tools and escalation      │
│  procedures.                                                                                                    │
│                                                                                                                 │
│  This document outlines the three most critical issues, their root causes, and a prioritized, three-point       │
│  action plan. Implementing these recommendations will significantly reduce resolution times, decrease ticket    │
│  volume, improve customer satisfaction, and create a valuable feedback loop for product development.            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **2. Key Findings: Critical Issues & Root Causes**                                                         │
│                                                                                                                 │
│  Our analysis identified three core areas of concern where process bottlenecks are directly creating poor       │
│  customer experiences.                                                                                          │
│                                                                                                                 │
│  *   **Critical Issue: Excessive Login Resolution Time (48 Hours)**                                             │
│      *   **Data:** Login problems are our most frequent support ticket (50 last quarter), but the critical      │
│  failure is the 48-hour average time to resolve them. This is the primary driver of customer frustration and    │
│  negative sentiment.                                                                                            │
│      *   **Root Cause:** Support agents cannot effectively diagnose these issues. They lack the tools to see    │
│  why a login failed and have difficulty reproducing the

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d60c6fe9-8898-43d5-871f-3fb62925aa7f                                                                     │
│  Agent: Executive Report Writer                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 18c4ab3c-5646-4187-9ab4-3171a130ae4b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **TO:** Chief Operating Officer                                                                  │
│  **FROM:** Executive Report Writer                                                                              │
│  **DATE:** October 26, 2023                                                                                     │
│  **SUBJECT:** Executive Summary: Analysis and Recommendations for Customer Support Process Improvement          │
│                                                                                                                 │
│  ### **1. Executive Overview**                                                                                  │
│                                                                                                                 │
│  This report summarizes a joint analysis of last quarter's customer support data and underlying operational     │
│  processes. The findings reveal that a few critical, preventable issues are driving the majority of our         │
│  customer support workload and negative sentiment. Our primary challenge is a **48-hour average resolution      │
│  time for login issues**, caused by significant gaps in our support team's diagnostic tools and escalation      │
│  procedures.                                                                                                    │
│                                                                                                                 │
│  This document outlines the three most critical issues, their root causes, and a prioritized, three-point       │
│  action plan. Implementing these recommendations will significantly reduce resolution times, decrease ticket    │
│  volume, improve customer satisfaction, and create a valuable feedback loop for product development.            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **2. Key Findings: Critical Issues & Root Causes**                                                         │
│                                                                                                                 │
│  Our analysis identified three core areas of concern where process bottlenecks are directly creating poor       │
│  customer experiences.                                                                                          │
│                                                                                                                 │
│  *   **Critical Issue: Excessive Login Resolution Time (48 Hours)**                                             │
│      *   **Data:** Login problems are our most frequent support ticket (50 last quarter), but the critical      │
│  failure is the 48-hour average time to resolve them. This is the primary driver of customer frustration and    │
│  negative sentiment.                                                                                            │
│      *   **Root Cause:** Support agents cannot effecti

--- Crew Execution Finished ---
--- Final Report for COO ---
**TO:** Chief Operating Officer
**FROM:** Executive Report Writer
**DATE:** October 26, 2023
**SUBJECT:** Executive Summary: Analysis and Recommendations for Customer Support Process Improvement

### **1. Executive Overview**

This report summarizes a joint analysis of last quarter's customer support data and underlying operational processes. The findings reveal that a few critical, preventable issues are driving the majority of our customer support workload and negative sentiment. Our primary challenge is a **48-hour average resolution time for login issues**, caused by significant gaps in our support team's diagnostic tools and escalation procedures.

This document outlines the three most critical issues, their root causes, and a prioritized, three-point action plan. Implementing these recommendations will significantly reduce resolution times, decrease ticket volume, improve customer satisfaction, and create a valuable fee